# Cats v/s Dogs kernel rewritten for practice

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import PIL
import zipfile
import os

# Define Global Variables

In [ ]:
# Path to zip file

path_to_train_zip = '../input/train.zip'
path_to_test_zip = '../input/test1.zip'
directory_to_extract = '.'

BATCH_SIZE = 256
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 1

train_imgs_path = './train'
test_imgs_path = './test1'

label_map = {
    'dog' : 1,
    'cat' : 0
}

inverse_label_map = {
    1 : 'dog',
    0 : 'cat'
}

# Extract the data

In [ ]:
with zipfile.ZipFile(path_to_train_zip, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract)
    
with zipfile.ZipFile(path_to_test_zip, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract)

# dataLoader

In [ ]:
class DVCLoader(torch.utils.data.Dataset):
    def __init__(self, path_to_images, label_map, dim_img):
        '''
        Provide link to images directory in path_to_images
        '''
        super().__init__()
        self.all_images = [f"{path_to_images}/{img}" for img in os.listdir(path_to_images)]
        self.labels = [label.split('.')[0] for label in os.listdir(path_to_images)]
        
        self.length = len(self.all_images)
        
        self.label_map = label_map
        self.transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((dim_img, dim_img)),
            torchvision.transforms.ToTensor(),            
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        '''
        Return PIL Image
        '''
        label_numeric = torch.tensor(self.label_map[self.labels[idx]], dtype = torch.float32)
        img_pil = PIL.Image.open(self.all_images[idx])
        img_final = self.transforms(img_pil)
        return (img_final, label_numeric)

# Data Splitting

In [ ]:
a = DVCLoader(train_imgs_path, label_map, dim_img = 250)
train_data, val_data = torch.utils.data.random_split(a, [15000, 10000])

# Data Loader

In [ ]:
train_dl = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True, num_workers = 8, pin_memory=True)
val_dl = torch.utils.data.DataLoader(val_data, batch_size = BATCH_SIZE, shuffle = True, num_workers = 8, pin_memory=True)

# Define Model

In [ ]:
class CVDClassifier(torch.nn.Module):
    '''
    cats v/s dogs
    '''
    def __init__(self):
        super().__init__()
        self.backbone = torchvision.models.squeezenet1_0(pretrained=True)
        # outputs 1x1000 dimensional vector
        self.out = torch.nn.Linear(1000, 1)
        # just outputs the logits, loss optimizer handled separately
        
    def forward(self, img_tensor):
        return self.out(self.backbone(img_tensor))

# Training Loop Parameters

In [ ]:
# set optimizer, loss, metric
model = CVDClassifier()
model = model.to(DEVICE)
loss_function = torch.nn.BCEWithLogitsLoss(reduction = 'mean').cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# # training loop
# for epoch in range(EPOCHS):
#     model.train()
#     for batch_idx, (img, labels) in enumerate(train_dl):
        
#         labels = labels.cuda()
        
#         output = model(img.cuda())
#         loss = loss_function(output, labels.view(-1, 1))
        
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        
#         # Validation
#         if epoch % 5 == 0 and batch_idx % 5 == 0:
#             with torch.no_grad():
#                 model.eval()
#                 img, label = next(iter(val_dl))
#                 output = model(img.cuda())
#                 loss = loss_function(output, labels.view(-1, 1).float())
#                 print(f"Loss EPOCH {epoch} BATCH {batch_idx} {loss.detach()}")

# Submission file

In [ ]:
submission = pd.DataFrame()
id_col = []
label_col = []
tfs = torchvision.transforms.Compose([
            torchvision.transforms.Resize((250, 250)),
            torchvision.transforms.ToTensor(),            
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])

model.eval()

for _img in os.listdir(test_imgs_path):
    img_id = int(_img.split('.')[0])
    img = PIL.Image.open(f"{test_imgs_path}/{_img}")
    img_tensor = tfs(img)
    with torch.no_grad():
        pred = int(model(img_tensor.unsqueeze(0).cuda()) > 0.5)
        
    id_col.append(img_id)
    label_col.append(pred)
    
    

# Final Submission file
submission['id'] = id_col
submission['label'] = label_col

SUBMISSION_IDENTIFIER = 'run_1_torch_squeezenet'
submission.to_csv(f'submission_{SUBMISSION_IDENTIFIER}.csv', index=False)

In [ ]:
!rm -r ./train
!rm -r ./test1